In [ ]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import re

In [ ]:
data = pd.DataFrame(columns=["title", "abstract", "source", "link", "topic", "time", "imglink"])

In [ ]:
#url dùng để thu thập dữ liệu từ báo Lao Động
u1 = "https://laodong.vn/xa-hoi?page="
u2 = "https://laodong.vn/the-thao?page="
u3 = "https://laodong.vn/van-hoa?page="
u4 = "https://laodong.vn/kinh-doanh?page="
u5 = "https://laodong.vn/the-gioi?page="
u6 = "https://laodong.vn/thoi-su?page="
u7 = "https://laodong.vn/cong-doan?page="
u8 = "https://laodong.vn/giai-tri?page="

In [ ]:
def laodong_topic(web_link):
    if "xa-hoi" in web_link:
        return "Xã hội"
    elif "the-thao" in web_link:
        return "Thể thao"
    elif "van-hoa" in web_link:
        return "Văn hóa"
    elif "giai-tri" in web_link:
        return "Giải trí"
    elif "kinh-doanh" in web_link:
        return "Kinh doanh"
    elif "the-gioi" in web_link:
        return "Thế giới"
    elif "thoi-su" in web_link:
        return "Thời sự"
    elif "cong-doan" in web_link:
        return "Công đoàn"
    else:
        return "Unknown category"

In [ ]:
from urllib.parse import urlparse, parse_qs

In [ ]:
#lấy tiêu đề
def find_all_articles_at_laodong(max_sl, web_link, data):
  for i in range(1, max_sl + 1):
    url = web_link + str(i)
    response = requests.get(url)
    soup = BeautifulSoup(response.content, "html.parser")
    all_articles = soup.find_all('article', class_='v4 p2c m002')
    for article in all_articles:
      title = article.select_one('a.link-title h2.title').get_text(strip=True)
      href = article.select_one('a.link-title')['href']
      if article.select_one('div.chapeau'):
        abstract = article.select_one('div.chapeau').get_text(strip=True)
      else:
        response2 = requests.get(href)
        soup2 = BeautifulSoup(response2.content, "html.parser")
        abstract = soup2.find('div', class_='chappeau').get_text(strip=True)
      response2 = requests.get(href)
      soup2 = BeautifulSoup(response2.content, "html.parser")
      time = soup2.find('span', class_ = 'time')
      if time:
        time = time.get_text(strip=True).split(',')[1].strip().split(' ')
        time = time[0] + " " + time[1]
      image_link = article.select_one('img')['src']
      parsed_url = urlparse(image_link)
      image_link = f"{parsed_url.scheme}://{parsed_url.netloc}{parsed_url.path}"
      data = data.append({"title" : title, "abstract": abstract, "source": "Báo Lao Động", "link" : href, "topic": laodong_topic(web_link), "time" : time, "imglink" : image_link}, ignore_index=True)
  return data

In [ ]:
#craft time
data = find_all_articles_at_laodong(20, u1, data)
data = find_all_articles_at_laodong(20, u2, data)
data = find_all_articles_at_laodong(20, u3, data)
data = find_all_articles_at_laodong(20, u4, data)
data = find_all_articles_at_laodong(20, u5, data)
data = find_all_articles_at_laodong(20, u6, data)
data = find_all_articles_at_laodong(20, u7, data)
data = find_all_articles_at_laodong(20, u8, data)
data = data.drop_duplicates()
data

<ipython-input-6-66367e76d71e>:26: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  data = data.append({"title" : title, "abstract": abstract, "source": "Báo Lao Động", "link" : href, "topic": laodong_topic(web_link), "time" : time, "imglink" : image_link}, ignore_index=True)
<ipython-input-6-66367e76d71e>:26: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  data = data.append({"title" : title, "abstract": abstract, "source": "Báo Lao Động", "link" : href, "topic": laodong_topic(web_link), "time" : time, "imglink" : image_link}, ignore_index=True)
<ipython-input-6-66367e76d71e>:26: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  data = data.append({"title" : title, "abstract": abstract, "source": "Báo Lao Động", "link" : href, "top

,title,abstract,source,link,topic,time,imglink
0,Đổi thay trên xã nông thôn mới nâng cao đầu ti...,"Xã Đắk Wer, huyện Đắk R'lấp là địa phương đầu ...",Báo Lao Động,https://laodong.vn/xa-hoi/doi-thay-tren-xa-non...,Xã hội,20/12/2023 11:42,https://media-cdn-v2.laodong.vn/storage/newspo...
1,Một người đàn ông nhảy cầu trong công viên tự tử,"Ngày 20.12, Công an TP Pleiku, tỉnh Gia Lai đa...",Báo Lao Động,https://laodong.vn/xa-hoi/mot-nguoi-dan-ong-nh...,Xã hội,20/12/2023 11:34,https://media-cdn-v2.laodong.vn/storage/newspo...
2,"Đà Nẵng công khai gần 30 đợt mở bán, cho thuê ...","Thông tin từ Sở Xây dựng Đà Nẵng ngày 20.12, t...",Báo Lao Động,https://laodong.vn/xa-hoi/da-nang-cong-khai-ga...,Xã hội,20/12/2023 11:32,https://media-cdn-v2.laodong.vn/storage/newspo...
3,Hiện trường vụ tai nạn giao thông giữa 2 ôtô t...,Vụ va chạm giao thông giữa 2 ôtô tải tại QL1A ...,Báo Lao Động,https://laodong.vn/ban-tin/hien-truong-vu-tai-...,Xã hội,20/12/2023 11:31,https://media-cdn-v2.laodong.vn/storage/newspo...
4,Dự báo tác động của không khí lạnh rất mạnh tr...,"Theo nhận định từ cơ quan khí tượng, không khí...",Báo Lao Động,https://laodong.vn/moi-truong/du-bao-tac-dong-...,Xã hội,20/12/2023 11:23,https://media-cdn-v2.laodong.vn/storage/newspo...
...,...,...,...,...,...,...,...
2380,Khả năng đăng quang Ca sĩ mặt nạ của Cú Tây Bắ...,"Cú Tây Bắc, Voi Bản Đôn, Ong Bây Bi là 3 masco...",Báo Lao Động,https://laodong.vn/giai-tri/kha-nang-dang-quan...,Giải trí,29/11/2023 21:08,https://media-cdn-v2.laodong.vn/storage/newspo...
2381,Hàng loạt sự kiện mừng Tết Dương lịch 2024 ở T...,"TPHCM - Bắn pháo hoa, lễ đếm ngược (Countdown)...",Báo Lao Động,https://laodong.vn/van-hoa-giai-tri/hang-loat-...,Giải trí,29/11/2023 20:33,https://media-cdn-v2.laodong.vn/storage/newspo...
2382,"Nhan sắc bất chấp thời gian của Thủy Tiên ""Vị ...","Dù đã từ giã màn ảnh rất nhiều năm, song cựu d...",Báo Lao Động,https://laodong.vn/giai-tri/nhan-sac-bat-chap-...,Giải trí,29/11/2023 20:10,https://media-cdn-v2.laodong.vn/storage/newspo...
2383,Ninh Dương Lan Ngọc bị nghi bắt chước Chi Pu t...,Khán giả phát hiện nhiều điểm tương đồng trong...,Báo Lao Động,https://laodong.vn/giai-tri/ninh-duong-lan-ngo...,Giải trí,29/11/2023 17:01,https://media-cdn-v2.laodong.vn/storage/newspo...


In [ ]:
data.to_csv('laodong.csv', index = False, encoding = 'utf-8-sig')

In [ ]:
u1 = "https://tuoitre.vn/the-gioi/trang-"
u2 = "https://tuoitre.vn/thoi-su/trang-"
u3 = "https://tuoitre.vn/phap-luat/trang-"
u4 = "https://tuoitre.vn/kinh-doanh/trang-"
u5 = "https://tuoitre.vn/cong-nghe/trang-"
u6 = "https://tuoitre.vn/xe/trang-"
u7 = "https://tuoitre.vn/du-lich/trang-"
u8 = "https://tuoitre.vn/nhip-song-tre/trang-"
u9 = "https://tuoitre.vn/van-hoa/trang-"
u10 = "https://tuoitre.vn/giai-tri/trang-"
u11 = "https://tuoitre.vn/the-thao/trang-"
u12 = "https://tuoitre.vn/giao-duc/trang-"
u13 = "https://tuoitre.vn/nha-dat/trang-"
u14 = "https://tuoitre.vn/suc-khoe/trang-"
u15 = "https://tuoitre.vn/gia-that/trang-"

In [ ]:
data2 = pd.DataFrame(columns=["title", "abstract", "source", "link", "topic", "time", " imglink"])

In [ ]:
def tuoitre_topic(web_link):
    if "the-gioi" in web_link:
        return "Thế giới"
    elif "thoi-su" in web_link:
        return "Thời sự"
    elif "phap-luat" in web_link:
        return "Pháp luật"
    elif "kinh-doanh" in web_link:
        return "Kinh doanh"
    elif "cong-nghe" in web_link:
        return "Công nghệ"
    elif "xe" in web_link:
        return "Xe"
    elif "du-lich" in web_link:
        return "Du lịch"
    elif "nhip-song-tre" in web_link:
        return "Nhịp sống trẻ"
    elif "van-hoa" or "giai-tri" in web_link:
        return "Văn hóa giải trí"
    elif "the-thao" in web_link:
        return "Thể thao"
    elif "giao-duc" in web_link:
        return "Giáo dục"
    elif "nha-dat" in web_link:
        return "Nhà đất"
    elif "suc-khoe" in web_link:
        return "Sức khỏe"
    elif "gia-that" in web_link:
        return "Giả - Thật"
    else:
        return "Unknown category"

In [ ]:
def find_all_articles_at_tuoitre(max_sl, web_link, data):
  for i in range(1, max_sl + 1):
    url = web_link + str(i) + '.htm'
    response = requests.get(url)
    soup = BeautifulSoup(response.content, "html.parser")
    all_articles = soup.select('div.box-category-content')
    for article in all_articles:
      if article.select_one('div.box-content-title h3 a'):
        title = article.select_one('div.box-content-title h3 a')['title']
        href = article.select_one('div.box-content-title h3 a')['href']
        if article.select_one('p.box-category-sapo'):
          abstract = article.select_one('p.box-category-sapo').get_text(strip=True)
        else:
          abstract = None
        data = data.append({"title" : title, "abstract": abstract, "source": "Báo Tuổi Trẻ", "link" : "https://tuoitre.vn"+href, "topic": tuoitre_topic(web_link)}, ignore_index=True)
      elif article.select_one('div.box-content-title h2 a'):
        title = article.select_one('div.box-content-title h2 a')['title']
        href = article.select_one('div.box-content-title h2 a')['href']
        if article.select_one('p.box-category-sapo'):
          abstract = article.select_one('p.box-category-sapo').get_text(strip=True)
        else:
          abstract = None
        data = data.append({"title" : title, "abstract": abstract, "source": "Báo Tuổi Trẻ", "link" : "https://tuoitre.vn"+href, "topic": tuoitre_topic(web_link)}, ignore_index=True)
      else:
        continue
    return data

In [ ]:
for u in [u1, u2, u3, u4, u5, u6, u7, u8, u9, u10, u11, u12, u13, u14, u15]:
    data2 = find_all_articles_at_tuoitre(1, u, data2)

<ipython-input-72-ff05ee577fa0>:23: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  data = data.append({"title" : title, "abstract": abstract, "source": "Báo Tuổi Trẻ", "link" : "https://tuoitre.vn"+href, "topic": tuoitre_topic(web_link)}, ignore_index=True)
<ipython-input-72-ff05ee577fa0>:15: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  data = data.append({"title" : title, "abstract": abstract, "source": "Báo Tuổi Trẻ", "link" : "https://tuoitre.vn"+href, "topic": tuoitre_topic(web_link)}, ignore_index=True)
<ipython-input-72-ff05ee577fa0>:15: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  data = data.append({"title" : title, "abstract": abstract, "source": "Báo Tuổi Trẻ", "link" : "https://tuoitre.vn"+href, "topic": tuoitre

In [ ]:
data2 = data2.drop_duplicates()
data2

,title,abstract,source,link,topic
0,Philippines muốn có quy tắc ứng xử với Việt Na...,Tổng thống Philippines Ferdinand Marcos Jr. ch...,Báo Tuổi Trẻ,https://tuoitre.vn/philippines-muon-co-quy-tac...,Thế giới
1,Giấc mộng Starship của SpaceX còn bao xa?,None,Báo Tuổi Trẻ,https://tuoitre.vn/giac-mong-starship-cua-spac...,Thế giới
2,Tổng thống đắc cử Argentina đối mặt lạm phát 3...,None,Báo Tuổi Trẻ,https://tuoitre.vn/tong-thong-dac-cu-argentina...,Thế giới
3,"Mùa đông càng gần, Nga sẽ tấn công Ukraine càn...",None,Báo Tuổi Trẻ,https://tuoitre.vn/mua-dong-cang-gan-nga-se-ta...,Thế giới
4,Tin tức thế giới 20-11: Nhóm Houthi nói đã bắt...,None,Báo Tuổi Trẻ,https://tuoitre.vn/tin-tuc-the-gioi-20-11-nhom...,Thế giới
...,...,...,...,...,...
205,Mạo danh tổng đài bảo hiểm xã hội lừa cước phí...,None,Báo Tuổi Trẻ,https://tuoitre.vn/mao-danh-tong-dai-bao-hiem-...,Văn hóa giải trí
206,Hội thảo Quân đoàn Thiết giáp Việt là lỗi dịch...,None,Báo Tuổi Trẻ,https://tuoitre.vn/hoi-thao-quan-doan-thiet-gi...,Văn hóa giải trí
207,"Siu Black còn sống nha, em vẫn sống, đang hát ...",None,Báo Tuổi Trẻ,https://tuoitre.vn/siu-black-con-song-nha-em-v...,Văn hóa giải trí
208,"Mang sổ đỏ đi tích hợp VNeID, sự thật hay chiê...",None,Báo Tuổi Trẻ,https://tuoitre.vn/mang-so-do-di-tich-hop-vnei...,Văn hóa giải trí


In [ ]:
data2.to_csv('tuoitre.csv', index = False, encoding = 'utf-8-sig')

In [ ]:
#craft data từ báo VnExpress
u1 = "https://vnexpress.net/thoi-su-p"
u3 = "https://vnexpress.net/the-gioi-p"
u4 = "https://vnexpress.net/kinh-doanh-p"
u5 = "https://vnexpress.net/bat-dong-san-p"
u6 = "https://vnexpress.net/khoa-hoc-p"
u7 = "https://vnexpress.net/giai-tri-p"
u8 = "https://vnexpress.net/the-thao-p"
u9 = "https://vnexpress.net/phap-luat-p"
u10 = "https://vnexpress.net/giao-duc-p"
u11 = "https://vnexpress.net/suc-khoe-p"
u12 = "https://vnexpress.net/doi-song-p"
u13 = "https://vnexpress.net/du-lich-p"
u14 = "https://vnexpress.net/so-hoa-p"
u15 = "https://vnexpress.net/oto-xe-may-p"

In [ ]:
def VNE_topic(web_link):
    if 'thoi-su' in web_link:
      return "Thời sự"
    elif 'goc-nhin' in web_link:
        return "Góc nhìn"
    elif 'the-gioi' in web_link:
        return "Thế giới"
    elif 'kinh-doanh' in web_link:
        return "Kinh doanh"
    elif 'bat-dong-san' in web_link:
        return "Bất động sản"
    elif 'khoa-hoc' in web_link:
        return "Khoa học"
    elif 'giai-tri' in web_link:
        return "Giải trí"
    elif 'the-thao' in web_link:
        return "Thể thao"
    elif 'phap-luat' in web_link:
        return "Pháp luật"
    elif 'giao-duc' in web_link:
        return "Giáo dục"
    elif 'suc-khoe' in web_link:
        return "Sức khỏe"
    elif 'doi-song' in web_link:
        return "Đời sống"
    elif 'du-lich' in web_link:
        return "Du lịch"
    elif 'so-hoa' in web_link:
        return "Công nghệ"
    elif 'oto-xe-may' in web_link:
        return "Xe"
    elif 'thu-gian' in web_link:
        return "Thư giãn"
    else:
        return ""

In [ ]:
data3 = pd.DataFrame(columns=["title", "abstract", "source", "link", "topic", "time", "imglink"])

In [ ]:
def find_all_articles_at_vne(max_sl, web_link, data):
  for i in range(1, max_sl + 1):
    url = web_link + str(i)
    response = requests.get(url)
    soup = BeautifulSoup(response.content, "html.parser")
    articles = soup.find_all('article', class_='item-news item-news-common thumb-left')
    for article in articles:
      if article.select_one('h3.title-news a'):
        imglink = article.select_one('img[itemprop="contentUrl"]')
        if imglink:
          if 'vcdn1' in imglink['src']:
            imglinkr = imglink['src']
            parsed_url = urlparse(imglinkr)
            imglinkr = f"{parsed_url.scheme}://{parsed_url.netloc}{parsed_url.path}"
          else:
            imglinkr = imglink['data-src']
            parsed_url = urlparse(imglinkr)
            imglinkr = f"{parsed_url.scheme}://{parsed_url.netloc}{parsed_url.path}"
        else:
          continue
        link = article.select_one('h3.title-news a')['href']
        title = article.select_one('h3.title-news a').text.strip()
        abstract = article.select_one('p.description a').find_all(text=True, recursive=False)
        abstract = ' '.join(abstract).strip()
        response2 = requests.get(link)
        soup2 = BeautifulSoup(response2.content, "html.parser")
        time = soup2.find('span', class_ = 'date')
        if time:
          time = time.get_text(strip=True)
          time = time.split(',')
          h = time[2].split(' ')
          ftime = time[1] + " " + h[1]
        else:
          continue
        data = data.append({"title" : title, "abstract": abstract, "source": "Báo VnExpress", "link" : link, "topic": VNE_topic(web_link), "time" : ftime, "imglink" : imglinkr}, ignore_index=True)
      else:
        continue
  return data

In [ ]:
for u in [u1, u3, u4, u5, u6, u7, u8, u9, u10, u11, u12, u13, u14, u15]:
    data3 = find_all_articles_at_vne(1, u, data3)

<ipython-input-6-db6ed3e5c824>:23: DeprecationWarning: The 'text' argument to find()-type methods is deprecated. Use 'string' instead.
  abstract = article.select_one('p.description a').find_all(text=True, recursive=False)
<ipython-input-6-db6ed3e5c824>:35: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  data = data.append({"title" : title, "abstract": abstract, "source": "Báo VnExpress", "link" : link, "topic": VNE_topic(web_link), "time" : ftime, "imglink" : imglinkr}, ignore_index=True)
<ipython-input-6-db6ed3e5c824>:23: DeprecationWarning: The 'text' argument to find()-type methods is deprecated. Use 'string' instead.
  abstract = article.select_one('p.description a').find_all(text=True, recursive=False)
<ipython-input-6-db6ed3e5c824>:35: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  data = data.append({"titl

KeyboardInterrupt: 

In [ ]:
data3 = data3.drop_duplicates()
data3

,title,abstract,source,link,topic,time,imglink
0,Ba bảo vật quốc gia tượng Chăm ở Đà Nẵng,"Ba hiện vật độc bản, niên đại thế kỷ 7-10, mô ...",Báo VnExpress,https://vnexpress.net/ba-bao-vat-quoc-gia-tuon...,Thời sự,20/1/2024 00:00,https://vcdn1-vnexpress.vnecdn.net/2024/01/19/...
1,Gần trăm nhà đầu tư quan tâm cải tạo chung cư ...,Sau khi thành phố ban hành đề án cải tạo chung...,Báo VnExpress,https://vnexpress.net/gan-tram-nha-dau-tu-quan...,Thời sự,19/1/2024 19:47,https://vcdn1-vnexpress.vnecdn.net/2024/01/19/...
2,Số hóa hiện vật tại Nhà trưng bày Hoàng Sa,300 hiện vật khẳng định chủ quyền Hoàng Sa của...,Báo VnExpress,https://vnexpress.net/so-hoa-hien-vat-tai-nha-...,Thời sự,19/1/2024 19:04,https://vcdn1-vnexpress.vnecdn.net/2024/01/19/...
3,"Hà Nội xử lý hơn 570 công chức, viên chức vi p...","Công an TP Hà Nội đã xử lý 575 đảng viên, công...",Báo VnExpress,https://vnexpress.net/ha-noi-xu-ly-hon-570-con...,Thời sự,19/1/2024 17:30,https://vcdn1-vnexpress.vnecdn.net/2024/01/19/...
4,2.024 drone sẽ trình diễn trong đêm giao thừa ...,TP Hà Nội sẽ xác lập kỷ lục Đông Nam Á với 2.0...,Báo VnExpress,https://vnexpress.net/2-024-drone-se-trinh-die...,Thời sự,19/1/2024 17:07,https://vcdn1-vnexpress.vnecdn.net/2024/01/19/...
...,...,...,...,...,...,...,...
318,U não chèn ép suýt chết sau một tháng đau đầu,"Chàng trai 21 tuổi nhức đầu, lơ mơ, ngủ li bì ...",Báo VnExpress,https://vnexpress.net/u-nao-chen-ep-suyt-chet-...,Sức khỏe,18/1/2024 14:36,https://vcdn1-suckhoe.vnecdn.net/2024/01/18/u-...
319,"Thuyên tắc ối sau sinh, sản phụ hai lần chạm c...",Sản phụ 34 tuổi ngưng tim ngưng thở khi vừa si...,Báo VnExpress,https://vnexpress.net/thuyen-tac-oi-sau-sinh-s...,Sức khỏe,18/1/2024 12:10,https://vcdn1-suckhoe.vnecdn.net/2024/01/18/ca...
320,Bì lợn bị tẩy trắng - món ăn tiềm ẩn độc tố,"Bì lợn giàu collagen, là thực phẩm được ưa chu...",Báo VnExpress,https://vnexpress.net/bi-lon-bi-tay-trang-mon-...,Sức khỏe,18/1/2024 09:29,https://vcdn1-suckhoe.vnecdn.net/2024/01/18/bu...
321,Cách ăn giúp mẹ hai con lấy lại vóc dáng thời ...,"Bảo Ly chọn cách tiêu thụ thực phẩm theo ""quy ...",Báo VnExpress,https://vnexpress.net/cach-an-giup-me-hai-con-...,Sức khỏe,18/1/2024 06:03,https://vcdn1-suckhoe.vnecdn.net/2024/01/17/ly...


In [ ]:
print(len(data3))

323


In [ ]:
data3.to_csv('vnexpress.csv', index = False, encoding = 'utf-8-sig')

In [ ]:
u1 = "https://vtc.vn/thoi-su-28/trang-"
u2 = "https://vtc.vn/kinh-te-29/trang-"
u3 = "https://vtc.vn/the-thao-34/trang-"
u4 = "https://vtc.vn/the-gioi-30/trang-"
u5 = "https://vtc.vn/giao-duc-31/trang-"
u6 = "https://vtc.vn/phap-luat-32/trang-"
u7 = "https://vtc.vn/giai-tri-33/trang-"
u8 = "https://vtc.vn/suc-khoe-35/trang-"
u9 = "https://vtc.vn/khoa-hoc-cong-nghe-82/trang-"
u10 = "https://vtc.vn/oto-xe-may-37/trang-"
u11 = "https://vtc.vn/phong-su-kham-pha-36/trang-"
u12 = "https://vtc.vn/song-ket-noi-123.html"

In [ ]:
def vtc_topic(web_link):
    if "thoi-su" in web_link:
        return "Thời sự"
    elif "kinh-te" in web_link:
        return "Kinh tế"
    elif "the-thao" in web_link:
        return "Thể thao"
    elif "the-gioi" in web_link:
        return "Thế giới"
    elif "giao-duc" in web_link:
        return "Giáo dục"
    elif "phap-luat" in web_link:
        return "Pháp luật"
    elif "giai-tri" in web_link:
        return "Giải trí"
    elif "suc-khoe" in web_link:
        return "Sức khỏe"
    elif "khoa-hoc-cong-nghe" in web_link:
        return "Khoa học công nghệ"
    elif "oto-xe-may" in web_link:
        return "Xe"
    elif "phong-su-kham-pha" in web_link:
        return "Phóng sự"
    else:
        return "Khám phá"

In [ ]:
data4 = pd.DataFrame(columns=["title", "abstract", "source", "link", "topic", "time", "imglink"])

In [ ]:
def find_all_articles_at_vtc(max_sl, web_link, data):
  for i in range(1, max_sl + 1):
    url = web_link + str(i) + ".html"
    response = requests.get(url)
    soup = BeautifulSoup(response.content, "html.parser")
    more_articles = soup.find_all('article', class_='clearfix distance')
    for article in more_articles:
      if article.select_one('h3.title-1 a'):
        title = article.select_one('h3.title-1 a')['title']
        link = article.select_one('h3.title-1 a')['href']
        abstract = article.select_one('p').get_text(strip=True)
        imglink = article.select_one('figure img')['data-src']
        time = article.select_one('footer span.time-update').text
        time = str(time).split(' ')
        time2 = time[1] + " " + time[0]
        data = data.append({"title" : title, "abstract": abstract, "source": "Báo VTC", "link" : "https://vtc.vn"+link, "topic": vtc_topic(web_link), "time" : time2, "imglink" : imglink}, ignore_index=True)
      else:
        continue
    more_more_articles = soup.find_all('article', class_='distance clearfix')
    for article in more_more_articles:
      if article.select_one('h3.title-1 a'):
        title = article.select_one('h3.title-1 a')['title']
        link = article.select_one('h3.title-1 a')['href']
        abstract = article.select_one('p').get_text(strip=True)
        imglink = article.select_one('figure img')['data-src']
        time = article.select_one('footer span.time-update').text
        time = str(time).split(' ')
        time2 = time[1] + " " + time[0]
        data = data.append({"title" : title, "abstract": abstract, "source": "Báo VTC", "link" : "https://vtc.vn"+link, "topic": vtc_topic(web_link), "time" : time2, "imglink" : imglink}, ignore_index=True)
      else:
        continue
  return data

In [ ]:
for u in [u1, u2, u3, u4, u5, u6, u7, u8, u9, u10, u11, u12]:
    data4 = find_all_articles_at_vtc(70, u, data4)

Kết quả truyền trực tuyến bị cắt bớt đến 5000 dòng cuối.
<ipython-input-37-17260e1ff2f0>:16: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  data = data.append({"title" : title, "abstract": abstract, "source": "Báo VTC", "link" : "https://vtc.vn"+link, "topic": vtc_topic(web_link), "time" : time2, "imglink" : imglink}, ignore_index=True)
<ipython-input-37-17260e1ff2f0>:29: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  data = data.append({"title" : title, "abstract": abstract, "source": "Báo VTC", "link" : "https://vtc.vn"+link, "topic": vtc_topic(web_link), "time" : time2, "imglink" : imglink}, ignore_index=True)
<ipython-input-37-17260e1ff2f0>:29: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  data = data.append({"title" : ti

In [ ]:
data4 = data4.drop_duplicates(subset = 'title', keep = 'last')
data4

,title,abstract,source,link,topic,time,imglink
30,Hải Phòng mạnh tay xử lý cán bộ gây khó trong ...,Chủ tịch UBND TP Hải Phòng yêu cầu kiên quyết ...,Báo VTC,https://vtc.vn/hai-phong-manh-tay-xu-ly-can-bo...,Thời sự,20/11/2023 16:28,https://cdn-i.vtcnews.vn/resize/CANbelY-rurH0d...
31,Ông Trần Hoàng Ngân giữ chức Trợ lý Bí thư Thà...,Ban Thường vụ Thành ủy TP.HCM công bố quyết đị...,Báo VTC,https://vtc.vn/ong-tran-hoang-ngan-giu-chuc-tr...,Thời sự,20/11/2023 15:45,https://cdn-i.vtcnews.vn/resize/CANbelY-rurkr_...
32,14 thuyền viên Bình Định gặp nạn trên biển đượ...,14 thuyền viên Bình Định sau khoảng 10 giờ gặp...,Báo VTC,https://vtc.vn/14-thuyen-vien-binh-dinh-gap-na...,Thời sự,20/11/2023 15:36,https://cdn-i.vtcnews.vn/resize/CANbelY-ruoi-v...
33,Ông Nguyễn Mạnh Cường giữ chức Trưởng Ban Dân ...,"Ông Nguyễn Mạnh Cường, Chánh Văn phòng Thành ủ...",Báo VTC,https://vtc.vn/ong-nguyen-manh-cuong-giu-chuc-...,Thời sự,20/11/2023 15:31,https://cdn-i.vtcnews.vn/resize/CANbelY-rupCZ0...
34,"Đấu giá thành công 5.421 biển số ô tô, trị giá...",Trong 153.104 biển số đẹp được đưa ra đấu giá ...,Báo VTC,https://vtc.vn/dau-gia-thanh-cong-5-421-bien-s...,Thời sự,20/11/2023 14:45,https://cdn-i.vtcnews.vn/resize/CANbelY-rup1e8...
...,...,...,...,...,...,...,...
26599,Trung Quốc tuyên bố thu hoạch được xà lách trồ...,"Theo Space, các phi hành gia thuộc sứ mệnh Thầ...",Báo VTC,https://vtc.vn/trung-quoc-tuyen-bo-thu-hoach-d...,Khám phá,07/11/2023 16:44,https://cdn-i.vtcnews.vn/resize/CANbelY-ruoqtN...
26600,Cuộc chạm trán UFO tại bệnh viện xuất hiện trê...,Cuộc chạm trán UFO ở Duncan vào năm 1970 trở t...,Báo VTC,https://vtc.vn/cuoc-cham-tran-ufo-tai-benh-vie...,Khám phá,07/11/2023 16:29,https://cdn-i.vtcnews.vn/resize/CANbelY-rupfX7...
26601,NASA: Vành đai sao Thổ sẽ 'biến mất' vào năm 2025,"Theo các nhà khoa học, thời gian để con người ...",Báo VTC,https://vtc.vn/nasa-vanh-dai-sao-tho-se-bien-m...,Khám phá,06/11/2023 20:41,https://cdn-i.vtcnews.vn/resize/CANbelY-ruouG3...
26602,NASA công bố hình ảnh 'đầu lâu khổng lồ' phát ...,NASA vừa công bố hình ảnh về miệng núi lửa trô...,Báo VTC,https://vtc.vn/nasa-cong-bo-hinh-anh-dau-lau-k...,Khám phá,06/11/2023 16:27,https://cdn-i.vtcnews.vn/resize/CANbelY-ruooGA...


In [ ]:
data4.to_csv('vtc.csv', index = False, encoding = 'utf-8-sig')

In [ ]:
data5 = pd.DataFrame(columns=["title", "abstract", "source", "link", "topic"])

In [ ]:
u1 = "https://dangcongsan.vn/thoi-su/p/"
u2 = "https://dangcongsan.vn/lanh-dao-dang-nha-nuoc/p/"
u3 = "https://dangcongsan.vn/xay-dung-dang/p/"
u4 = "https://dangcongsan.vn/tu-tuong-van-hoa/p/"
u5 = "https://dangcongsan.vn/kinh-te/p/"
u6 = "https://dangcongsan.vn/xa-hoi/p/"
u7 = "https://dangcongsan.vn/noi-hay-dung/p/"
u8 = "https://dangcongsan.vn/cung-ban-luan/p/"

In [ ]:
def dcs_topic(web_link):
    if "thoi-su" in web_link:
        return "Thời sự"
    elif "lanh-dao-dang-nha-nuoc" in web_link:
        return "Lãnh đạo Đảng, Nhà nước"
    elif "xay-dung-dang" in web_link:
        return "Xây dựng Đảng"
    elif "tu-tuong-van-hoa" in web_link:
        return "Văn hóa"
    elif "kinh-te" in web_link:
        return "Kinh tế"
    elif "xa-hoi" in web_link:
        return "Xã hội"
    elif "noi-hay-dung" in web_link:
        return "Nơi hay dựng"
    elif "cung-ban-luan" in web_link:
        return "Cùng bàn luận"
    else:
        return "Unknown category"

In [ ]:
def find_all_articles_at_dcs(max_sl, web_link, data):
  for i in range(1, max_sl + 1):
    url = web_link + str(i)
    response = requests.get(url)
    soup = BeautifulSoup(response.content, "html.parser")
    articles = soup.find_all('article')
    for article in articles:
      if article.select_one('div.h3.h3cate'):
        title = article.select_one('div.h3.h3cate').get_text(strip=True)
        link = article.select_one('div.col0.box-thumbnail a')['href']
        abstract = article.select_one('p.pctent').get_text(strip=True)
        data = data.append({"title" : title, "abstract": abstract, "source": "Báo Đảng Cộng Sản", "link" : link, "topic": dcs_topic(web_link)}, ignore_index=True)
      else:
        continue
  return data

In [ ]:
for u in [u1, u2, u3, u4, u5, u6, u7, u8]:
    data5 = find_all_articles_at_dcs(50, u, data5)

Kết quả truyền trực tuyến bị cắt bớt đến 5000 dòng cuối.
<ipython-input-123-8552561707d7>:12: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  data = data.append({"title" : title, "abstract": abstract, "source": "Báo Đảng Cộng Sản", "link" : link, "topic": vcs_topic(web_link)}, ignore_index=True)
<ipython-input-123-8552561707d7>:12: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  data = data.append({"title" : title, "abstract": abstract, "source": "Báo Đảng Cộng Sản", "link" : link, "topic": vcs_topic(web_link)}, ignore_index=True)
<ipython-input-123-8552561707d7>:12: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  data = data.append({"title" : title, "abstract": abstract, "source": "Báo Đảng Cộng Sản", "link" : link, "topic": vcs

In [ ]:
data5 = data5.drop_duplicates()
data5

,title,abstract,source,link,topic
0,"Bảo tồn, phát huy giá trị các di sản văn hóa","Phó Chủ tịch Thường trực Quốc hội đề nghị, Hội...",Báo Đảng Cộng Sản,https://dangcongsan.vn/thoi-su/bao-ton-phat-hu...,Thời sự
1,Phó Chủ tịch Quốc hội Trần Quang Phương tiếp T...,"Chiều tối 20/11, tại Nhà Quốc hội, Phó Chủ tịc...",Báo Đảng Cộng Sản,https://dangcongsan.vn/thoi-su/pho-chu-tich-qu...,Thời sự
2,"Phát triển ngành Halal Việt Nam bài bản, chuyê...",(ĐCSVN) - Thứ trưởng Ngoại giao Nguyễn Minh Hằ...,Báo Đảng Cộng Sản,https://dangcongsan.vn/thoi-su/phat-trien-ngan...,Thời sự
3,Thúc đẩy mạnh mẽ hơn nữa công cuộc phòng chống...,(ĐCSVN) - Theo Thứ trưởng Bộ Y tế Trần Văn Thu...,Báo Đảng Cộng Sản,https://dangcongsan.vn/y-te/thuc-day-manh-me-h...,Thời sự
4,"Bí thư Thành ủy TP Hồ Chí Minh thăm, chúc mừng...","(ĐCSVN) - Thăm, chúc mừng các nhà giáo tiêu bi...",Báo Đảng Cộng Sản,https://dangcongsan.vn/giao-duc/bi-thu-thanh-u...,Thời sự
...,...,...,...,...,...
11967,"Đại biểu Quốc hội bày tỏ lo ngại việc miễn, gi...","(ĐCSVN) - Việc miễn, giảm thuế đã được ban hàn...",Báo Đảng Cộng Sản,https://dangcongsan.vn/day-manh-cai-cach-tu-ph...,Cùng bàn luận
11968,Nghĩ về chất vấn và trả lời chất vấn tại Quốc hội,(ĐCSVN) - Dù hoạt động chất vấn và trả lời chấ...,Báo Đảng Cộng Sản,https://dangcongsan.vn/tieu-diem/nghi-ve-chat-...,Cùng bàn luận
11969,Phương án nào để bảo vệ cây xanh Hà Nội?,"(ĐCSVN) – Những ngày qua, thông tin Hà Nội dịc...",Báo Đảng Cộng Sản,https://dangcongsan.vn/cung-ban-luan/phuong-an...,Cùng bàn luận
11970,Cách mạng công nghiệp 4.0: Nhìn từ sự cạnh tra...,"(ĐCSVN) – Trong điều kiện hiện nay, khi cuộc c...",Báo Đảng Cộng Sản,https://dangcongsan.vn/cung-ban-luan/cach-mang...,Cùng bàn luận


In [ ]:
data5.to_csv('dcs.csv', index = False, encoding = 'utf-8-sig')